# Plotting AMSR-E data from a HDF

In [1]:
from pyhdf.SD import SD, SDC
from pathlib import Path

# Test should run from root
import os

os.chdir("..")

In [2]:
from omegaconf import OmegaConf
from multiearth.api import extract_assets
from multiearth.config import ConfigSchema

cfg = OmegaConf.load("config/nsidc-amsr.yaml")
cfg_schema = OmegaConf.structured(ConfigSchema)
cfg = OmegaConf.merge(cfg_schema, cfg)

# Constrain the test to only pull one item and one band
for pvdr in cfg.providers:
    for coll in pvdr.collections:
        coll.max_items = 1
        coll.assets = ["data"]

success = extract_assets(cfg)
print("Successfully extracted assets." if success else "Asset extraction failed.")

/Users/raymond/opt/anaconda3/envs/multiearth/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
16:04:00 INFO - Extraction details for provider ProviderKey.EARTHDATA with collection AE_5DSno: 
id: AE_MoSno
assets:
- data
outdir: null
datetime: null
aoi_file: null
max_items: 1

16:04:00 INFO - Extraction details for provider ProviderKey.EARTHDATA with collection AE_DySno: 
id: AE_MoSno
assets:
- data
outdir: null
datetime: null
aoi_file: null
max_items: 1

16:04:00 INFO - Extraction details for provider ProviderKey.EARTHDATA with collection AE_MoSno: 
id: AE_MoSno
assets:
- data
outdir: null
datetime: null
aoi_file: null
max_items: 1

16:04:01 INFO - EarthData Provider returned 1 items for AE_5DSno for datetime 2000-05-01/2005-03-01
16:04:01 INFO - EarthData Provider returned 1 items for AE_DySno for datetim

Successfully extracted assets.


Let's load our HDF files we obtained from MultiEarth

In [3]:
# Find the correct NC file and load it
# We avoid hardcoding the path here in case the API changes down the road

fpath = Path(cfg["default_collection"]["outdir"])
collections = ["AE_5DSno", "AE_DySno", "AE_MoSno"]
hdf_file_paths = [
    list((fpath / collection).glob("**/*.hdf")) for collection in collections
]

# We know there is only one file pulled, assert
assert (
    len(hdf_file_paths) == 3
), f"There were {len(hdf_files)} files pulled. `expected 3` failed to constrain."

hdf_files = [SD(str(hdf_file_fpath[0]), SDC.READ) for hdf_file_fpath in hdf_file_paths]

In [4]:
for hdf in hdf_files:
    dataset = hdf.datasets()
    print(dataset)

{'SWE_NorthernPentad': (('YDim:Northern Hemisphere', 'XDim:Northern Hemisphere'), (721, 721), 21, 0), 'Flags_NorthernPentad': (('YDim:Northern Hemisphere', 'XDim:Northern Hemisphere'), (721, 721), 21, 1), 'SWE_SouthernPentad': (('YDim:Southern Hemisphere', 'XDim:Southern Hemisphere'), (721, 721), 21, 2), 'Flags_SouthernPentad': (('YDim:Southern Hemisphere', 'XDim:Southern Hemisphere'), (721, 721), 21, 3)}
{'SWE_NorthernDaily': (('YDim:Northern Hemisphere', 'XDim:Northern Hemisphere'), (721, 721), 21, 0), 'Flags_NorthernDaily': (('YDim:Northern Hemisphere', 'XDim:Northern Hemisphere'), (721, 721), 21, 1), 'SWE_SouthernDaily': (('YDim:Southern Hemisphere', 'XDim:Southern Hemisphere'), (721, 721), 21, 2), 'Flags_SouthernDaily': (('YDim:Southern Hemisphere', 'XDim:Southern Hemisphere'), (721, 721), 21, 3)}
{'SWE_NorthernMonth': (('YDim:Northern Hemisphere', 'XDim:Northern Hemisphere'), (721, 721), 21, 0), 'Flags_NorthernMonth': (('YDim:Northern Hemisphere', 'XDim:Northern Hemisphere'), (72